# Setting up env
- CUDA: 11.3
- CUDNN: 8.6.0
- Python: 3.9.5
- Pytorch: 1.12.1

In [2]:
import os
import copy
import time
import glob
import h5py
import torch
import monai
import shutil
import tempfile
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from tqdm import tqdm_gui
from monai.transforms import (
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
    Resized,
    RandSpatialCropd,
    CenterSpatialCropd,
    NormalizeIntensityd
)
import matplotlib.pyplot as plt
import torch.nn.functional as F
from monai.data import CacheDataset
from scipy.ndimage import _nd_image
from torch.utils.data import Dataset
from monai.config import print_config
from monai.networks.layers import Norm
from monai.utils import MetricReduction
from torch.utils.data import DataLoader
from monai.apps import download_and_extract
from monai.utils import first, set_determinism
from monai.inferers import sliding_window_inference
from scipy.ndimage import distance_transform_edt as distance
from monai.metrics import compute_meandice, compute_hausdorff_distance
from monai.metrics.utils import do_metric_reduction, ignore_background
device = torch.device("cuda:0")
print(device)

cuda:0


# Loss Functions

## Not in Loop

In [3]:
def hd_loss_2D(seg_soft, gt, seg_dtm, gt_dtm):
    """
    compute huasdorff distance loss for binary segmentation
    input: seg_soft: softmax results,  shape=(x,y)
           gt: ground truth, shape=(x,y)
           seg_dtm: segmentation distance transform map; shape=(x,y)
           gt_dtm: ground truth distance transform map; shape=(x,y)
    output: boundary_loss; sclar
    """

    delta_s = (seg_soft - gt) ** 2
    s_dtm = seg_dtm ** 2
    g_dtm = gt_dtm ** 2
    dtm = s_dtm + g_dtm
    multipled = torch.einsum('xy, xy->xy', delta_s, dtm)
    hd_loss = multipled.mean()

    return hd_loss

In [4]:
def hd_loss_3D(seg_soft, gt, seg_dtm, gt_dtm):
    """
    compute huasdorff distance loss for binary segmentation
    input: seg_soft: softmax results,  shape=(x,y,z)
           gt: ground truth, shape=(x,y,z)
           seg_dtm: segmentation distance transform map; shape=(x,y,z)
           gt_dtm: ground truth distance transform map; shape=(x,y,z)
    output: boundary_loss; sclar
    """

    delta_s = (seg_soft - gt) ** 2
    s_dtm = seg_dtm ** 2
    g_dtm = gt_dtm ** 2
    dtm = s_dtm + g_dtm
    multipled = torch.einsum('xyz, xyz->xyz', delta_s, dtm)
    hd_loss = multipled.mean()

    return hd_loss

## In Loop

In [5]:
# def hd_loss_2D_loop(seg_soft, gt, seg_dtm, gt_dtm):
#     """
#     compute huasdorff distance loss for binary segmentation
#     input: seg_soft: softmax results,  shape=(b,2,x,y,z)
#            gt: ground truth, shape=(b,x,y,z)
#            seg_dtm: segmentation distance transform map; shape=(b,2,x,y,z)
#            gt_dtm: ground truth distance transform map; shape=(b,2,x,y,z)
#     output: boundary_loss; sclar
#     """

#     delta_s = (seg_soft - gt) ** 2
#     s_dtm = seg_dtm ** 2
#     g_dtm = gt_dtm ** 2
#     dtm = s_dtm + g_dtm
#     multipled = torch.einsum('bcxy, bcxy->bcxy', delta_s, dtm)
#     hd_loss = multipled.mean()

#     return hd_loss

# Prepare other stuff for timing

## Create test cases

In [6]:
num_range_2D = np.arange(2.0, 513.0, 21.25)
num_range_3D = np.arange(2.0, 129.0, 5.25)
test_cases_2D = []
test_cases_3D = []
sizes_2D = []
sizes_3D = []
# test_cases_2D_loop = []
# test_cases_3D_loop = []
# sizes_2D_loop = []
# sizes_3D_loop = []
for i in num_range_2D:
    n = int(i)
    _size = n**2
    #2D
    sizes_2D.append(_size)
    # non loop
    pred = torch.from_numpy(np.random.rand(n,n).astype(np.float32))
    mask = torch.from_numpy(np.random.randint(2, size=(n,n)).astype(np.float32))
    test_cases_2D.append([pred, mask])

# for n in 16 : 16 : 512
#     print("2DLoop $n --> ")
#     #2D
#     push!(sizes_2D_loop, n^2)
#     # loop
#     x = rand(Float32, n, n, 1, 1)
#     mask = Float32.(rand([0, 1], n, n, 1, 1))
#     push!(test_cases_2D_loop, deepcopy([x, mask]))
# end
for i in num_range_3D:
    n = int(i)
    _size = n**3
    #3D
    sizes_3D.append(_size)
    # non loop
    pred = torch.from_numpy(np.random.rand(n,n,n).astype(np.float32))
    mask = torch.from_numpy(np.random.randint(2, size=(n,n,n)).astype(np.float32))
    test_cases_3D.append([pred, mask])
# for n in 16 : 16 : 128
#     print("3DLoop $n --> ")
#     #3D
#     push!(sizes_3D_loop, n^3)
#     # loop
#     x = rand(Float32, n, n, n, 1, 1)
#     mask = Float32.(rand([0, 1], n, n, n, 1, 1))
#     push!(test_cases_3D_loop, deepcopy([x, mask]))
# end

## Create 2D U-net

In [7]:
def conv2d(stride, in_dim, out_dim):
    return nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1)

def tran2d(stride, in_dim, out_dim):
    return nn.ConvTranspose2d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1, output_padding=1)

def conv12d(in_dim, out_dim):
    return nn.Sequential(
        conv2d(1, in_dim, out_dim),
        nn.BatchNorm2d(out_dim),
        nn.LeakyReLU())

def conv22d(in_dim, out_dim):
    return nn.Sequential(
        conv2d(2, in_dim, out_dim),
        nn.BatchNorm2d(out_dim),
        nn.LeakyReLU())

def conv32d(in_dim, out_dim):
    return nn.Sequential(
        conv2d(1, in_dim, out_dim),
        nn.Softmax(dim=1))
        # nn.Sigmoid())

def tran22d(in_dim, out_dim):
    return nn.Sequential(
        tran2d(2, in_dim, out_dim),
        nn.BatchNorm2d(out_dim),
        nn.LeakyReLU())

def concat2d(layer1, layer2):
    return torch.cat([layer1, layer2], dim=1)

class unet2D_new(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(unet2D_new, self).__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        # Contracting layers
        self.l1 = conv12d(in_dim, 8)
        self.l2 = nn.Sequential(conv22d(8, 16), conv12d(16, 16))
        self.l3 = nn.Sequential(conv22d(16, 32), conv12d(32, 32))
        self.l4 = nn.Sequential(conv22d(32, 64), conv12d(64, 64))
        self.l5 = nn.Sequential(conv22d(64, 128), conv12d(128, 128))

        # Expanding layers
        self.l6 = tran22d(128, 64)
        self.l7 = nn.Sequential(conv12d(128, 64), tran22d(64, 32))
        self.l8 = nn.Sequential(conv12d(64, 32), tran22d(32, 16))
        self.l9 = nn.Sequential(conv12d(32, 16), tran22d(16, 8))
        self.l10 = nn.Sequential(conv32d(8, out_dim))

    def forward(self, x):
        # Contracting layers
        l1_out = self.l1(x)
        l2_out = self.l2(l1_out)
        l3_out = self.l3(l2_out)
        l4_out = self.l4(l3_out)
        l5_out = self.l5(l4_out)

        # Expanding layers
        l6_out = self.l6(l5_out)
        l7_out = self.l7(concat2d(l4_out, l6_out))
        l8_out = self.l8(concat2d(l3_out, l7_out))
        l9_out = self.l9(concat2d(l2_out, l8_out))
        out = self.l10(l9_out)

        return out

Test Model

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 96
x = torch.Tensor(1, 1, image_size, image_size)
# x.to(device)
print("in size: {}".format(x.size()))

m_org = unet2D_new(1, 2)
o = m_org(x)
print("out size: {}".format(o.size()))

in size: torch.Size([1, 1, 96, 96])
out size: torch.Size([1, 2, 96, 96])


## Create 3D U-net

In [9]:
def conv(stride, in_dim, out_dim):
    return nn.Conv3d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1)

def tran(stride, in_dim, out_dim):
    return nn.ConvTranspose3d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1, output_padding=1)

def conv1(in_dim, out_dim):
    return nn.Sequential(
        conv(1, in_dim, out_dim),
        nn.BatchNorm3d(out_dim),
        nn.LeakyReLU())

def conv2(in_dim, out_dim):
    return nn.Sequential(
        conv(2, in_dim, out_dim),
        nn.BatchNorm3d(out_dim),
        nn.LeakyReLU())

def conv3(in_dim, out_dim):
    return nn.Sequential(
        conv(1, in_dim, out_dim),
        nn.Softmax(dim=1))
        # nn.Sigmoid())

def tran2(in_dim, out_dim):
    return nn.Sequential(
        tran(2, in_dim, out_dim),
        nn.BatchNorm3d(out_dim),
        nn.LeakyReLU())

def concat(layer1, layer2):
    return torch.cat([layer1, layer2], dim=1)

class unet3D_new(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(unet3D_new, self).__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        # Contracting layers
        self.l1 = conv1(in_dim, 8)
        self.l2 = nn.Sequential(conv2(8, 16), conv1(16, 16))
        self.l3 = nn.Sequential(conv2(16, 32), conv1(32, 32))
        self.l4 = nn.Sequential(conv2(32, 64), conv1(64, 64))
        self.l5 = nn.Sequential(conv2(64, 128), conv1(128, 128))

        # Expanding layers
        self.l6 = tran2(128, 64)
        self.l7 = nn.Sequential(conv1(128, 64), tran2(64, 32))
        self.l8 = nn.Sequential(conv1(64, 32), tran2(32, 16))
        self.l9 = nn.Sequential(conv1(32, 16), tran2(16, 8))
        self.l10 = nn.Sequential(conv3(8, out_dim))

    def forward(self, x):
        # Contracting layers
        l1_out = self.l1(x)
        l2_out = self.l2(l1_out)
        l3_out = self.l3(l2_out)
        l4_out = self.l4(l3_out)
        l5_out = self.l5(l4_out)

        # Expanding layers
        l6_out = self.l6(l5_out)
        l7_out = self.l7(concat(l4_out, l6_out))
        l8_out = self.l8(concat(l3_out, l7_out))
        l9_out = self.l9(concat(l2_out, l8_out))
        out = self.l10(l9_out)

        return out

Test model

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 96
x = torch.Tensor(1, 1, image_size, image_size, image_size).cuda()
x.to(device)
print("in size: {}".format(x.size()))

m = unet3D_new(1, 2).cuda()
o = m(x)
print("out size: {}".format(o.size()))

in size: torch.Size([1, 1, 96, 96, 96])
out size: torch.Size([1, 2, 96, 96, 96])


## Distance Transform

In [11]:
# A working DT
def compute_dtm_loop(img_gt, out_shape):
    """
    compute the distance transform map of foreground in binary mask
    input: segmentation, shape = (batch_size, num_channels, x, y, z)
    output: the foreground Distance Map (SDM) 
    dtm(x) = 0; x in segmentation boundary
             inf|x-y|; x in segmentation
    """

    fg_dtm = np.zeros(out_shape)

    for b in range(out_shape[0]): # each batch
        for c in range(out_shape[1]): # each channel
            # 3D DT
            posmask = img_gt[b][c]
            if posmask.any():
                posdis = distance(1 - posmask)
                fg_dtm[b][c] = posdis

    return fg_dtm

# Timing

## Not in loop

### 2D

In [12]:
not_loop_2d_Scipy_cpu_hd_cpu_min = []
not_loop_2d_Scipy_cpu_hd_cpu_std = []

not_loop_2d_Scipy_cpu_hd_gpu_min = []
not_loop_2d_Scipy_cpu_hd_gpu_std = []

for (idx, case) in enumerate(test_cases_2D):
    pred, mask = copy.deepcopy(case)
    # cpu -> gpu
    pred_gpu, mask_gpu = pred.cuda(), mask.cuda()
    
    #---Start testing---#

    # tfm = Scipy, tfm_device = cpu, hd_device = cpu
    Scipy_cpu_hd_cpu = []
    for eval in range(1000):
        t1 = time.perf_counter_ns()
        # DT
        pred_dtm = torch.from_numpy(distance(pred))
        mask_dtm = torch.from_numpy(distance(mask))
        loss = hd_loss_2D(pred, mask, pred_dtm, mask_dtm)
        # Loss
        t2 = time.perf_counter_ns()
        Scipy_cpu_hd_cpu.append(t2-t1)
    # tfm = Scipy, tfm_device = cpu, hd_device = gpu
    Scipy_cpu_hd_gpu = []
    for eval in range(1000):
        t1 = time.perf_counter_ns()
        # DT
        pred_cpu, mask_cpu = pred_gpu.to('cpu'), mask_gpu.to('cpu')
        pred_dtm = torch.from_numpy(distance(pred_cpu))
        mask_dtm = torch.from_numpy(distance(mask_cpu))
        pred_dtm_gpu, mask_dtm_gpu = pred_dtm.cuda(), mask_dtm.cuda()
        loss = hd_loss_2D(pred_gpu, mask_gpu, pred_dtm_gpu, mask_dtm_gpu)
        # Loss
        t2 = time.perf_counter_ns()
        Scipy_cpu_hd_gpu.append(t2-t1)

    #---Finished testing---#
    not_loop_2d_Scipy_cpu_hd_cpu_min.append(np.min(Scipy_cpu_hd_cpu))
    not_loop_2d_Scipy_cpu_hd_cpu_std.append(np.std(Scipy_cpu_hd_cpu))
    
    not_loop_2d_Scipy_cpu_hd_gpu_min.append(np.min(Scipy_cpu_hd_gpu))
    not_loop_2d_Scipy_cpu_hd_gpu_std.append(np.std(Scipy_cpu_hd_gpu))

    print("size = {}, Scipy_cpu = {:0.2f}ms, Scipy_gpu = {:0.2f}ms".format(sizes_2D[idx], not_loop_2d_Scipy_cpu_hd_cpu_min[-1]*1e-6, not_loop_2d_Scipy_cpu_hd_gpu_min[-1]*1e-6))


size = 4, Scipy_cpu = 0.09ms, Scipy_gpu = 0.28ms
size = 529, Scipy_cpu = 0.12ms, Scipy_gpu = 0.31ms
size = 1936, Scipy_cpu = 0.20ms, Scipy_gpu = 0.40ms
size = 4225, Scipy_cpu = 0.33ms, Scipy_gpu = 0.56ms
size = 7569, Scipy_cpu = 0.53ms, Scipy_gpu = 0.75ms
size = 11664, Scipy_cpu = 0.77ms, Scipy_gpu = 1.17ms
size = 16641, Scipy_cpu = 1.06ms, Scipy_gpu = 1.56ms
size = 22500, Scipy_cpu = 1.39ms, Scipy_gpu = 1.97ms
size = 29584, Scipy_cpu = 1.80ms, Scipy_gpu = 2.48ms
size = 37249, Scipy_cpu = 2.35ms, Scipy_gpu = 3.44ms
size = 45796, Scipy_cpu = 2.89ms, Scipy_gpu = 4.10ms
size = 55225, Scipy_cpu = 3.47ms, Scipy_gpu = 4.80ms
size = 66049, Scipy_cpu = 4.50ms, Scipy_gpu = 5.87ms
size = 77284, Scipy_cpu = 5.29ms, Scipy_gpu = 6.78ms
size = 89401, Scipy_cpu = 6.01ms, Scipy_gpu = 7.71ms
size = 102400, Scipy_cpu = 7.15ms, Scipy_gpu = 10.76ms
size = 116964, Scipy_cpu = 8.20ms, Scipy_gpu = 10.16ms
size = 131769, Scipy_cpu = 13.68ms, Scipy_gpu = 13.04ms
size = 147456, Scipy_cpu = 13.44ms, Scipy_gpu = 

Save Data

In [13]:
data2D = {'size': sizes_2D, 
'Scipy_cpu_hd_cpu_min': not_loop_2d_Scipy_cpu_hd_cpu_min, 
'Scipy_cpu_hd_cpu_std': not_loop_2d_Scipy_cpu_hd_cpu_std, 
'Scipy_cpu_hd_gpu_min': not_loop_2d_Scipy_cpu_hd_gpu_min, 
'Scipy_cpu_hd_gpu_std': not_loop_2d_Scipy_cpu_hd_gpu_std}
dataframe2D = pd.DataFrame(data2D)
dataframe2D.to_csv("C:/Users/wenbl13/Desktop/Ashwin-Timing/distance-transforms/HD_2D_Not_Loop_Jan_12.csv")

### 3D

In [14]:
not_loop_3d_Scipy_cpu_hd_cpu_min = []
not_loop_3d_Scipy_cpu_hd_cpu_std = []

not_loop_3d_Scipy_cpu_hd_gpu_min = []
not_loop_3d_Scipy_cpu_hd_gpu_std = []

for (idx, case) in enumerate(test_cases_3D):
    pred, mask = copy.deepcopy(case)
    # cpu -> gpu
    pred_gpu, mask_gpu = pred.cuda(), mask.cuda()
    
    #---Start testing---#

    # tfm = Scipy, tfm_device = cpu, hd_device = cpu
    Scipy_cpu_hd_cpu = []
    for eval in range(1000):
        t1 = time.perf_counter_ns()
        # DT
        pred_dtm = torch.from_numpy(distance(pred))
        mask_dtm = torch.from_numpy(distance(mask))
        loss = hd_loss_3D(pred, mask, pred_dtm, mask_dtm)
        # Loss
        t2 = time.perf_counter_ns()
        Scipy_cpu_hd_cpu.append(t2-t1)
    # tfm = Scipy, tfm_device = cpu, hd_device = gpu
    Scipy_cpu_hd_gpu = []
    for eval in range(1000):
        t1 = time.perf_counter_ns()
        # DT
        pred_cpu, mask_cpu = pred_gpu.to('cpu'), mask_gpu.to('cpu')
        pred_dtm = torch.from_numpy(distance(pred_cpu))
        mask_dtm = torch.from_numpy(distance(mask_cpu))
        pred_dtm_gpu, mask_dtm_gpu = pred_dtm.cuda(), mask_dtm.cuda()
        loss = hd_loss_3D(pred_gpu, mask_gpu, pred_dtm_gpu, mask_dtm_gpu)
        # Loss
        t2 = time.perf_counter_ns()
        Scipy_cpu_hd_gpu.append(t2-t1)

    #---Finished testing---#
    not_loop_3d_Scipy_cpu_hd_cpu_min.append(np.min(Scipy_cpu_hd_cpu))
    not_loop_3d_Scipy_cpu_hd_cpu_std.append(np.std(Scipy_cpu_hd_cpu))

    not_loop_3d_Scipy_cpu_hd_gpu_min.append(np.min(Scipy_cpu_hd_gpu))
    not_loop_3d_Scipy_cpu_hd_gpu_std.append(np.std(Scipy_cpu_hd_gpu))

    print("size = {}, Scipy_cpu = {:0.2f}ms, Scipy_gpu = {:0.2f}ms".format(sizes_3D[idx], not_loop_3d_Scipy_cpu_hd_cpu_min[-1]*1e-6, not_loop_3d_Scipy_cpu_hd_gpu_min[-1]*1e-6))


size = 8, Scipy_cpu = 0.10ms, Scipy_gpu = 0.35ms
size = 343, Scipy_cpu = 0.13ms, Scipy_gpu = 0.41ms
size = 1728, Scipy_cpu = 0.26ms, Scipy_gpu = 0.59ms
size = 4913, Scipy_cpu = 0.56ms, Scipy_gpu = 0.95ms
size = 12167, Scipy_cpu = 1.24ms, Scipy_gpu = 1.72ms
size = 21952, Scipy_cpu = 2.16ms, Scipy_gpu = 3.13ms
size = 35937, Scipy_cpu = 3.70ms, Scipy_gpu = 4.66ms
size = 54872, Scipy_cpu = 6.25ms, Scipy_gpu = 7.33ms
size = 85184, Scipy_cpu = 9.94ms, Scipy_gpu = 11.40ms
size = 117649, Scipy_cpu = 15.77ms, Scipy_gpu = 17.06ms
size = 157464, Scipy_cpu = 22.83ms, Scipy_gpu = 23.17ms
size = 205379, Scipy_cpu = 30.72ms, Scipy_gpu = 30.66ms
size = 274625, Scipy_cpu = 42.17ms, Scipy_gpu = 42.69ms
size = 343000, Scipy_cpu = 54.18ms, Scipy_gpu = 53.19ms
size = 421875, Scipy_cpu = 67.63ms, Scipy_gpu = 65.48ms
size = 512000, Scipy_cpu = 84.37ms, Scipy_gpu = 81.47ms
size = 636056, Scipy_cpu = 103.28ms, Scipy_gpu = 100.10ms
size = 753571, Scipy_cpu = 121.49ms, Scipy_gpu = 118.08ms
size = 884736, Scipy_c

Save Data

In [15]:
data3D = {'size': sizes_3D, 
'Scipy_cpu_hd_cpu_min': not_loop_3d_Scipy_cpu_hd_cpu_min, 
'Scipy_cpu_hd_cpu_std': not_loop_3d_Scipy_cpu_hd_cpu_std, 
'Scipy_cpu_hd_gpu_min': not_loop_3d_Scipy_cpu_hd_gpu_min, 
'Scipy_cpu_hd_gpu_std': not_loop_3d_Scipy_cpu_hd_gpu_std}
dataframe3D = pd.DataFrame(data3D)
dataframe3D.to_csv("C:/Users/wenbl13/Desktop/Ashwin-Timing/distance-transforms/HD_3D_Not_Loop_Jan_12.csv")